In [1]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import TreebankWordTokenizer

import matplotlib
import sklearn
from IPython.core.display import display, HTML

######################################################################################################################

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

get_ipython().run_line_magic('matplotlib', 'inline')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid", font_scale=1.9)

######################################################################################################################

# TO WORK WITH
import pandas as pd
import numpy as np
from numpy import set_printoptions

# HIDE WARNINGS
import warnings
warnings.filterwarnings('ignore')

# PREPROCESSING & MODEL SELECTION
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import randint
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression, LinearRegression, RidgeCV, LassoCV, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import SCORERS
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

# print(SCORERS.keys())

# PLOTTING
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook")
get_ipython().run_line_magic('matplotlib', 'inline')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn import tree
from graphviz import Source
from matplotlib.pylab import rcParams
import matplotlib.lines as mlines
from yellowbrick.classifier import PrecisionRecallCurve
from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ConfusionMatrix
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.cluster import InterclusterDistance
import plotly.express as px
import scipy.cluster.hierarchy as sch
from sklearn.metrics import classification_report


# STANDARD MODELS
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# ENSEMBLE
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

# XGBOOST
from xgboost import XGBClassifier
import xgboost as xgb

# CLUSTERING
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

# PICKLE
from pickle import dump
from pickle import load




In [35]:

# data

In [57]:
    data = pd.read_csv("listings_8.csv")
    data=data.dropna()
    
    data=data[data["number_of_reviews"]>=1]
    data=data[data["guests_included"]>=1]

    
    for categorical_feature in ['neighbourhood_cleansed', 'room_type']:
        data = pd.concat([data, pd.get_dummies(data[categorical_feature])], axis=1)
        
#     data.head()
    
    data= data.drop(['neighbourhood_cleansed', 'property_type', 'room_type','description','host_about_bool',
                    'amenities',"house_rules","review_scores_cleanliness","review_scores_rating",'review_scores_accuracy',              
                    'review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_value'                
                    ],axis=1)
    
    
    scaler=MinMaxScaler(feature_range=(0,1))
    
    data[['security_deposit']] = scaler.fit_transform(data[['security_deposit']])
    data[['cleaning_fee']] = scaler.fit_transform(data[['cleaning_fee']])
    data[['host_response_rate']] = scaler.fit_transform(data[['host_response_rate']])
    data[['price_per_night']] = scaler.fit_transform(data[['price_per_night']])
    
    data[['accommodates']] = scaler.fit_transform(data[['accommodates']])
    data[['bathrooms']] = scaler.fit_transform(data[['bathrooms']])
    data[['bedrooms']] = scaler.fit_transform(data[['bedrooms']])
    data[['beds']] = scaler.fit_transform(data[['beds']])
    data[['guests_included']] = scaler.fit_transform(data[['guests_included']])
    data[['extra_people']] = scaler.fit_transform(data[['extra_people']])
    data[['availability_365']] = scaler.fit_transform(data[['availability_365']])
    data[['number_of_reviews']] = scaler.fit_transform(data[['number_of_reviews']])

    data.info()
    data["host_response_rate"]
    data.describe()
    
    
    
    # WE STILL HAVE ID TO LINK IT TO THE CALENDAR TABLE!

#     y = data["price_per_night"] # Target variable (price_per_night)
#     X_wnei = data.drop(["price_per_night"],axis=1)
    
#     # Creating new DF without neighborhood names
#     X_wnei.to_csv("X_wnei.csv", index=False)
#     data.to_csv("data.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2181 entries, 0 to 2767
Data columns (total 77 columns):
id                                  2181 non-null int64
transit_bool                        2181 non-null bool
nopets_bool                         2181 non-null bool
nosmok_bool                         2181 non-null bool
fewdays_response_time               2181 non-null bool
1day_response_time                  2181 non-null bool
1hour_response_time                 2181 non-null bool
fewhours_response_time              2181 non-null bool
host_response_rate                  2181 non-null float64
host_is_superhost                   2181 non-null bool
host_identity_verified              2181 non-null bool
is_location_exact                   2181 non-null bool
accommodates                        2181 non-null float64
bathrooms                           2181 non-null float64
bedrooms                            2181 non-null float64
beds                                2181 non-null float

0       0.0
1       0.0
2       0.0
4       0.0
5       0.0
       ... 
2759    0.0
2760    0.0
2762    0.0
2765    0.0
2767    0.0
Name: host_response_rate, Length: 2181, dtype: float64

,id,host_response_rate,accommodates,bathrooms,bedrooms,beds,price_per_night,security_deposit,cleaning_fee,guests_included,...,Roslindale,Roxbury,South Boston,South Boston Waterfront,South End,West End,West Roxbury,Entire home/apt,Private room,Shared room
count,2.181000e+03,2181.000000,2181.000000,2181.000000,2181.000000,2181.000000,2181.000000,2181.000000,2181.000000,2181.000000,...,2181.000000,2181.000000,2181.000000,2181.000000,2181.000000,2181.000000,2181.000000,2181.000000,2181.000000,2181.000000
mean,7.379566e+06,0.931206,0.143359,0.203958,0.252545,0.184778,0.183305,0.112342,0.169829,0.043276,...,0.019716,0.045851,0.055479,0.022467,0.094911,0.010087,0.014672,0.598349,0.382852,0.018799
std,4.461995e+06,0.182743,0.124907,0.085582,0.156126,0.117389,0.134397,0.175832,0.179629,0.088180,...,0.139054,0.209209,0.228966,0.148230,0.293159,0.099950,0.120264,0.490344,0.486194,0.135845
min,3.353000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.701108e+06,0.960000,0.066667,0.166667,0.200000,0.111111,0.078408,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.403927e+06,1.000000,0.066667,0.166667,0.200000,0.111111,0.156815,0.000000,0.116667,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,1.157962e+07,1.000000,0.200000,0.166667,0.400000,0.222222,0.247286,0.189474,0.266667,0.076923,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
max,1.484305e+07,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
